## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer

In [2]:
df = pd.read_csv ('/datasets/insurance.csv')

In [3]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
df.duplicated().sum()

153

In [5]:
df.describe ()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

#### Изучив данные видим, что имеются 153 дубликата, пропуски отсутствуют, медиана и среднее практически равны, значит маловероятно наличие выбросов, минимальные и максимальные значения соответствуют реальным, что еще раз доказывает что выбросов нет. Для нашей задачи дубликаты не являются проблемой, поэтому удалять их не станем.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** Линейная регрессия в качестве предсказания скалярно умножает вектор признаков на вектор весов а затем к этому произведению прибавляет величину сдвига предсказания, исходя из этого при умножении на обратную матрицу расстояния между признаками изменится одинаково не изменив обобщаемость модели.  
В качестве обоснования мы используем модель линейной регрессии, которую сначала методом кросс-валидации оценим на исходных данных, а затем произведем умножение признаков на обратимую матрицу и снова протестиуем модель.

In [7]:
matrix = np.random.normal(0.5, 0.3, 16).reshape(4,4) 

In [8]:
matrix

array([[-0.3556796 ,  0.24267735,  0.8505968 ,  0.3857374 ],
       [ 0.87860814,  0.60182979, -0.37014255,  1.01902691],
       [-0.01650558,  0.23637966,  0.24229853,  0.18472661],
       [ 0.57246505,  1.01231351,  0.13927406,  0.4798564 ]])

In [9]:
# Проверяем матрицу на обратимость
np.linalg.inv (matrix)

array([[ 10.51448848,   0.14663238, -40.64365597,   6.88270737],
       [ -5.90439923,  -0.56103037,  21.13399698,  -2.19807229],
       [  8.40257106,  -0.27179971, -28.5529702 ,   4.81452122],
       [ -2.52645213,   1.08751569,  12.19016748,  -2.88734098]])

In [10]:
features = df.drop (['Страховые выплаты'], axis= 1)
display(features.shape)
target = df['Страховые выплаты']
display(target.shape)

(5000, 4)

(5000,)

In [11]:
R2 = make_scorer(r2_score)

In [12]:
# Линейная модель на базовых данных
np.mean(cross_val_score (LinearRegression (), features, target, cv=5, n_jobs = -1, scoring = R2))

0.4231137691590773

In [13]:
new_features = features.values @ matrix

In [14]:
# Линейная модель после умножения на обратимую матрицу
np.mean(cross_val_score (LinearRegression (), new_features, target, cv=5, n_jobs = -1, scoring = R2))

0.4231137691590761

### Вывод:  
#### Как мы видим, в не зависимости от рандомного создания обратимой матрицы, средняя оценка модели линейной регрессии отличается настолько мало, что их можно признать одинаковыми.

## 3. Алгоритм преобразования

**Алгоритм**

В качестве алгоритма кодировки предлагаю использование алгоритма минимума-максимума.

Обозначения:
- $x$ — элемент массива
- $MIN$ — минимальное значение в массиве
- $MAX$ — максимальное значение в массиве

$$
f(x) = (x - MIN(x)) / (MAX(x) - MIN(x))
$$

**Обоснование**

Качество линейной регресии не изменится по той же причине, почему оно не изменилось при умножении на обратимую матрицу. Расстояния между всеми точками изменится одинаково.

## 4. Проверка алгоритма

In [15]:
# Зададим функцию для кодирования признаков
def min_max_scale(values):
    feature = []
    for i in range(values.shape[0]):
        feature.append((values[i] - values.min()) / (values.max() - values.min()))
    return feature

In [16]:
columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
scaled_features = pd.DataFrame(min_max_scale (features.values), columns = columns)
display(features)
display(scaled_features)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


,Пол,Возраст,Зарплата,Члены семьи
0,0.000013,0.000519,0.627848,0.000013
1,0.000000,0.000582,0.481013,0.000013
2,0.000000,0.000367,0.265823,0.000000
3,0.000000,0.000266,0.527848,0.000025
4,0.000013,0.000354,0.330380,0.000000
...,...,...,...,...
4995,0.000000,0.000354,0.451899,0.000025
4996,0.000000,0.000430,0.663291,0.000013
4997,0.000000,0.000253,0.429114,0.000025
4998,0.000013,0.000278,0.413924,0.000038


In [17]:
print("Качество модели до преобразования:", np.mean(cross_val_score (LinearRegression (), features, target, cv = 5, n_jobs = -1, scoring = R2)))
print("Качество модели после преобразования:", np.mean(cross_val_score (LinearRegression (), scaled_features, target, cv = 5, n_jobs = -1, scoring = R2)))

Качество модели до преобразования: 0.4231137691590773
Качество модели после преобразования: 0.42311376915907817


### Вывод:  
#### Как видим, после нашего скалирования (шифрования) конфиденциальность данных обеспечена.  
#### В тоже время качество модели линейной регресии не изменилось.